In [2]:
# Install liblinear: sudo apt-get install python-liblinear
# from liblinearutil import *
import numpy as np

In [3]:
def read_problem(file_name):
    y = []
    x = []
    feature_len = 22
    for line in open(file_name):
        tmp = line.split(' ',1)
        y = y + [int(tmp[0])]
        vec = np.zeros(feature_len)
        for each in tmp[1].split():
            index,val = each.split(':')
            vec[int(index)-1] = float(val)
        x.append(vec)
    y = np.array(y)
    x = np.array(x)
    return y,x

# Gradient ascent step for dual variables

In [11]:
def get_tp_tn(x,y,w):
    h = x.dot(w)
    ind = np.where(h>0)
    y1 = -1*np.ones_like(y)
    y1[ind] = 1
    tp = sum(np.multiply(1+y,1+y1)/4)
    tn = sum(np.multiply(1-y,1-y1)/4)
    return tp,tn

def get_projection(alpha,beta):
    if alpha>0 and beta>0 and alpha*beta>=1.0/4:
        return alpha,beta
    a = 16
    b = -16*beta
    c = 0
    d = 4*alpha
    e = -1
    D0 = c*c - 3*b*d + 12*a*e
    D1 = 2*c*c*c-9*b*c*d+27*b*b*e+27*a*d*d-72*a*c*e
    p = (8*a*c-3*b*b)/(8*a*a)
    q = (b*b*b-4*a*b*c+8*a*a*d)
    Q = np.power((D1+np.sqrt(D1*D1-4*D0*D0*D0))/2.0,1/3)
    s = 0.5*np.sqrt(-2/3*p+1/(3*a)*(Q+D0/Q))
    y1 = -float(b)/(4*a) - s + 0.5*np.sqrt(-4*s*s-2*p+q/s)
    y2 = -float(b)/(4*a) - s - 0.5*np.sqrt(-4*s*s-2*p+q/s)
    y3 = -float(b)/(4*a) + s + 0.5*np.sqrt(-4*s*s-2*p+q/s)
    y4 = -float(b)/(4*a) + s - 0.5*np.sqrt(-4*s*s-2*p+q/s)
    x1 = 1/(4*y1)
    if x1>=1/4 and y1>=1/4:
        return x1,y1
    x2 = 1/(4*y2)
    if x2>=1/4 and y2>=1/4:
        return x2,y2
    x3 = 1/(4*y3)
    if x3>=1/4 and y3>=1/4:
        return x3,y3
    x4 = 1/(4*y4)
    if x4>=1/4 and y4>=1/4:
        return x4,y4
    return 1,1

def gradient_ascent(x,y,w,a,b,alpha=0.1):
    n = x.shape[0]
    n_pos = len([i for i in range(n) if y[i]==1])
    n_neg = n-n_pos
    ind_pos = [i for i in range(n) if y[i]>0]
    ind_neg = [i for i in range(n) if y[i]<0]
    tpr,tnr = get_tp_tn(x,y,w)
    tpr /= float(n_pos)
    tnr /= float(n_neg)
    print('TPR =',tpr,'TNR =',tnr)
    for i in range(n):
        g = 0
#         alpha = 1/float(i+1)
        if i in ind_pos:
            try:
                tmp = 1 - (2*n_pos)/(a*n)*(x[i].dot(w))
            except:
                tmp = 0
            if tmp > 0:
                g += 1.0/n_pos
            g = g-1.0/n_pos+0 # \Delta*'(a,b)=0
            a = a + alpha*g
        else:
            try:
                tmp = 1 + (2*n_neg)/(b*n)*(x[i].dot(w))
            except:
                tmp = 0
            if tmp > 0:
                g += 1.0/n_neg
            g = g-1.0/n_neg+0
            b = b + alpha*g
        # Projection to a,b \in R+ \intersect ab>1/4
        a,b = get_projection(a,b)
    return a,b

In [4]:
w = np.random.uniform(low=-1,high=1,size=(22,))
a,b = 1.0,1.0
y,x = read_problem('./data/ijcnn1.tr')
a,b=gradient_ascent(x,y,w,a,b)
print(a,b)

(35000, 3415, 31585)
('TPR =', 0.5537335285505125, 'TNR =', 0.50463827766344782)
(1.0, 0.9999833748500325)


## Liblinear for primal variable w

In [6]:
# customized liblinear
import sys
sys.path.insert(0, "/home/debojyoti/opt/liblinear-2.1")
from ppython import liblinear
from ppython.liblinear import *
from ppython.liblinearutil import *

In [12]:
def modifyx(x,y,a,b,n,n_pos,n_neg):
    scale_pos = float(2*n_pos)/(a*n)
    scale_neg = float(2*n_neg)/(b*n)
    for i in range(len(y)):
        if y[i]==1:
            x[i].update((key,val*scale_pos) for key,val in x[i].items())
        else:
            x[i].update((key,val*scale_neg) for key,val in x[i].items())
    return x
# init section
a,b = 1,1
y_lst,x_lst = read_problem('./data/ijcnn1.tr') # x as numpy list
n_dim = len(x_lst[0])
w = np.random.uniform(low=-1,high=1,size=(n_dim,))
y_orig,x_orig = svm_read_problem('./data/ijcnn1.tr') # x in liblinear compatible dictionary format
n = len(y)
n_pos = len([i for i in range(n) if y[i]==1])
n_neg = n-n_pos
c = 5.0
# Iterative section: Gradient Ascent & Liblinear
for i in range(10):
    a,b = gradient_ascent(x_lst,y_lst,w,a,b) #Gradient ascent
    c1 = float(a)/n_pos
    c2 = float(b)/n_neg
    param = parameter('-s 3 -w1 {} -w-1 {} -c {}'.format(c1,c2,c))
    x = modifyx(x_orig,y_orig,a,b,n,n_pos,n_neg)
    prob = problem(y_orig,x)
    model = train(prob,param)
    # Get model parameters
    w = model.get_decfun()[0]
    print(a,b)
#     print w

('TPR =', 0.65739385065885803, 'TNR =', 0.3847395915782808)
(1.0, 0.9881747665031206)
('TPR =', 0.0, 'TNR =', 1.0)
(1.0, 0.9322874782335318)
('TPR =', 0.0, 'TNR =', 1.0)
(1.0, 0.9322874782335318)
('TPR =', 0.0, 'TNR =', 1.0)
(1.0, 0.9322874782335318)
('TPR =', 0.0, 'TNR =', 1.0)
(1.0, 0.9322874782335318)
('TPR =', 0.0, 'TNR =', 1.0)
(1.0, 0.9322874782335318)
('TPR =', 0.0, 'TNR =', 1.0)
(1.0, 0.9322874782335318)
('TPR =', 0.0, 'TNR =', 1.0)
(1.0, 0.9322874782335318)
('TPR =', 0.0, 'TNR =', 1.0)
(1.0, 0.9322874782335318)
('TPR =', 0.0, 'TNR =', 1.0)
(1.0, 0.9322874782335318)


In [53]:
# y_orig,x_orig = svm_read_problem('./data/ijcnn1.tr')
t = x_orig[0]
print t
t.update((x,y*10) for x,y in t.items())
print t

{6: 10.0, 11: -7.3185400000000005, 12: 1.73431, 13: 0.0, 14: 0.0027, 15: 0.11684, 16: -0.11052, 17: 0.24452000000000002, 18: 0.08337, 19: 0.013240000000000002, 20: 0.25544, 21: -0.40728, 22: -0.0081}
{6: 100.0, 11: -73.1854, 12: 17.3431, 13: 0.0, 14: 0.027000000000000003, 15: 1.1684, 16: -1.1052, 17: 2.4452000000000003, 18: 0.8337, 19: 0.13240000000000002, 20: 2.5544000000000002, 21: -4.0728, 22: -0.08099999999999999}


In [8]:
y,x = svm_read_problem('./data/ijcnn1.tr')
prob = problem(y,x)
# Compute parameters
n = len(y)
n_pos = len([i for i in range(n) if y[i]==1])
n_neg = n-n_pos
c1 = a/n_pos
c2 = b/n_neg
param = parameter('-w-1 {} -c {}'.format(c2,c1))
model = train(prob,param)
# Get model parameters
model.get_decfun()

([0.020088400051177527,
  0.02078000240895638,
  0.017383681282139466,
  0.01922073585258815,
  0.01871566162727098,
  0.018957471615453476,
  0.020564558992360284,
  0.019204518613350992,
  0.018384635918062716,
  0.019919263327922897,
  0.0004097609048048318,
  0.04549050018405845,
  -0.00019739460456936307,
  -0.0003677184740392728,
  0.0003408332898644388,
  0.0007132671938060524,
  -0.014374657235245919,
  -0.01787257671927265,
  -0.011937403539873225,
  0.006379924826240279,
  0.0016604175632452852,
  0.0034085857729773757],
 0.0)

In [10]:
# def gradient_ascent_batch(x,y,w,alpha=0.1,ep=0.0001,maxiter=100):
#     n = x.shape[0]
#     n_pos = len([i for i in range(n) if y[i]==1])
#     n_neg = n-n_pos
#     a,b = 0.1,0.1
#     ind_pos = [i for i in range(n) if y[i]>0]
#     ind_neg = [i for i in range(n) if y[i]<0]
#     x_pos = x[ind_pos]
#     x_neg = x[ind_neg]
#     tpr,tnr = get_tp_tn(x,y,w)
#     tpr /= n_pos
#     tnr /= n_neg
#     print 'TPR =',tpr,'TNR =',tnr
#     for i in range(maxiter):
#         # Positive instances
#         tmp = 1 - (2*n_pos)/(a*n)*x_pos.dot(w)
#         j = np.where(tmp>0)
#         tmp = np.zeros_like(tmp)
#         tmp[j]=1/n_pos
#         a += alpha * (sum(tmp)-1+tpr)
#         # Negative instances
#         tmp = 1 + (2*n_neg)/(b*n)*x_neg.dot(w)
#         j = np.where(tmp>0)
#         tmp = np.zeros_like(tmp)
#         tmp[j]=1/n_neg
#         b += alpha * (sum(tmp)-1+tnr)
#         print a,b